In [2]:
import boto3

from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir


man_voice_id = "Joey"
woman_voice_id = "Joanna"
boy_voice_id = "Matthew"
girl_voice_id = "Ivy"

# text to speech
def tts(txt, voice_id, output_file_name="speech.mp3", is_upload_to_s3=True):
    bucket_name = "aigc-bj-team1"
    polly_client = boto3.client("polly")
    s3_client = boto3.client("s3")
    try:
        response = polly_client.synthesize_speech(Text=txt, OutputFormat="mp3", VoiceId=voice_id)
    except (BotoCoreError, ClientError) as error:
        print(error)
        sys.exit(-1)
    if "AudioStream" in response:
        with closing(response["AudioStream"]) as stream:
            if is_upload_to_s3:
                output = os.path.join(gettempdir(), output_file_name)
                try:
                    with open(output, "wb") as file:
                        file.write(stream.read())
                        s3_client.upload_file(output, bucket_name, output_file_name)
                        s3_output_link = f"s3://{bucket_name}/{output_file_name}"
                        print(f"Written to {s3_output_link}")
                        return s3_output_link
                except IOError as error:
                    print(error)
                    sys.exit(-1)
            else:
                try:
                    with open(output_file_name, "wb") as file:
                        file.write(stream.read())
                        print(f"Written to {output_file_name}")
                        return output_file_name
                except IOError as error:
                    print(error)
                    sys.exit(-1)
            
    else:
        print("Could not stream audio")
        sys.exit(-1)
    

In [33]:
tts('Hello, world!', man_voice_id)

Written to s3://aigc-bj-team1/speech.mp3


's3://aigc-bj-team1/speech.mp3'

In [28]:
!aws s3 cp s3://aigc-bj-team1/speech.mp3 ./speech.mp3

download: s3://aigc-bj-team1/speech.mp3 to ./speech.mp3        


In [29]:
from IPython.display import Audio

# Replace 'path_to_your_mp3.mp3' with the path to your mp3 file
audio_file = 'speech.mp3'

Audio(audio_file)